In [50]:
import wrangle as w
import clustering_feature_engineering as c

# making clusters:
# tabular data
import pandas as pd
import numpy as np
# visualizations
import matplotlib.pyplot as plt
import seaborn as sns
# sklearn:
# the thing that will do the clustering:
from sklearn.cluster import KMeans
# the thing that will do the scaling:
from sklearn.preprocessing import MinMaxScaler
# the thing that will/would do a train-test split:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import AgglomerativeClustering
from sklearn import cluster
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import silhouette_score

In [11]:
df, train, validate, test = w.wrangle_data()

Opening data from local file.


In [12]:
train_scaled, validate_scaled, test_scaled = scale_data(train, validate, test, columns_to_scale=['customer_age',
                                                                                                    'unit_cost', 
                                                                                                     'unit_price',
                                                                                                     'cost',
                                                                                                     'revenue',
                                                                                                     'profit'])

In [18]:
train_scaled.drop('date', axis=1, inplace=True)
validate_scaled.drop('date', axis=1, inplace=True)
test_scaled.drop('date', axis=1, inplace=True)


In [20]:
train_scaled.isna().sum()

year                              0
month                             0
customer_age                      0
customer_gender                   0
country                           0
state                             0
product_category                  0
sub_category                      0
quantity                          0
unit_cost                         0
unit_price                        0
cost                              0
revenue                           0
profit                            0
sub_category_Bike Stands          0
sub_category_Bottles and Cages    0
sub_category_Caps                 0
sub_category_Cleaners             0
sub_category_Fenders              0
sub_category_Gloves               0
sub_category_Helmets              0
sub_category_Hydration Packs      0
sub_category_Jerseys              0
sub_category_Mountain Bikes       0
sub_category_Road Bikes           0
sub_category_Shorts               0
sub_category_Socks                0
sub_category_Tires and Tubes

In [4]:
# def k_means_clustering(df):
#     # Drop date column from dataset
#     df.drop('date', axis=1, inplace=True)

#     # Encode categorical columns
#     le = LabelEncoder()
#     df['customer_gender'] = le.fit_transform(df['customer_gender'])
#     df['country'] = le.fit_transform(df['country'])
#     df['state'] = le.fit_transform(df['state'])
#     df['product_category'] = le.fit_transform(df['product_category'])
#     df['sub_category'] = le.fit_transform(df['sub_category'])
#     df['month'] = le.fit_transform(df['month'])

#     # Standardizes data
#     df = (df - df.mean()) / df.std()

#     # Feature Selection
#     data = df[['customer_age','sub_category', 'cost']]  # Select relevant columns

#     # Standardization
#     scaler = StandardScaler()
#     X = scaler.fit_transform(data)

#     # Clustering Algorithm (K-means)
#     k = 5  # Number of clusters
#     kmeans = KMeans(n_clusters=k)
#     labels = kmeans.fit_predict(X)

#     # Cluster Analysis
#     # Analyze the resulting clusters by examining their characteristics
#     cluster_centers = kmeans.cluster_centers_

#     # Print the mean values of the selected features for each cluster
#     for cluster in range(k):
#         cluster_data = data[labels == cluster]
#         cluster_mean = cluster_data.mean()
#         print(f"Cluster {cluster + 1} Mean:")
#         print(cluster_mean)
#         print()
#     # Print silhouette_score for clusters     
#     s_score = silhouette_score(X, labels)
#     print(f"Silhouette Score: {s_score:.3f}")

#     # Adding target column containing the labels
#     df['clusters'] = labels

#     # returns df with target column
#     return df
    
# # Execute with the following statement    
# # k_means_clustering(df)

In [54]:
def k_means_clustering(k):
    
    df, train, validate, test = w.wrangle_data()


    modeling_feats = ['customer_age','sub_category_Bike Stands',
       'sub_category_Bottles and Cages', 'sub_category_Caps',
       'sub_category_Cleaners', 'sub_category_Fenders', 'sub_category_Gloves',
       'sub_category_Helmets', 'sub_category_Hydration Packs',
       'sub_category_Jerseys', 'sub_category_Mountain Bikes',
       'sub_category_Road Bikes', 'sub_category_Shorts', 'sub_category_Socks',
       'sub_category_Tires and Tubes', 'sub_category_Touring Bikes',
       'sub_category_Vests', 'revenue'] 

    df_clustering = df[modeling_feats]

    # Standardization
    columnTransformer = ColumnTransformer([('scaler', StandardScaler(), 
                                            ['customer_age', 'revenue'])], 
                                          remainder="passthrough")
    X = columnTransformer.fit_transform(df_clustering)

    # Clustering Algorithm (K-means)
    kmeans = KMeans(n_clusters=k, random_state=123)
    labels = kmeans.fit_predict(X)
    
    # Cluster Analysis
    # Analyze the resulting clusters by examining their characteristics
    cluster_centers = kmeans.cluster_centers_

    # Print the mean values of the selected features for each cluster
    for cluster in range(k):
        cluster_data = df_clustering[labels == cluster]
        cluster_mean = cluster_data.mean()
        print(f"Cluster {cluster + 1} Mean:")
        print(cluster_mean)
        print()
    # Print silhouette_score for clusters     
    s_score = silhouette_score(X,labels)
    print(f"Silhouette Score: {s_score:.3f}")

    df['clusters'] = labels
    
    # returns df with target column
    return df

In [40]:
df, train, validate, test = w.wrangle_data()


modeling_feats = ['customer_age','sub_category_Bike Stands',
       'sub_category_Bottles and Cages', 'sub_category_Caps',
       'sub_category_Cleaners', 'sub_category_Fenders', 'sub_category_Gloves',
       'sub_category_Helmets', 'sub_category_Hydration Packs',
       'sub_category_Jerseys', 'sub_category_Mountain Bikes',
       'sub_category_Road Bikes', 'sub_category_Shorts', 'sub_category_Socks',
       'sub_category_Tires and Tubes', 'sub_category_Touring Bikes',
       'sub_category_Vests', 'revenue'] 

df_clustering = df[modeling_feats]

# Standardization
scaler = StandardScaler()
X = scaler.fit_transform(df_clustering[['customer_age', 'revenue']])

Opening data from local file.


array([[-0.66436297, -0.72202168],
       [-0.66436297, -0.79261247],
       [-0.66436297, -0.8496281 ],
       ...,
       [ 0.14551813,  1.27895564],
       [ 0.14551813,  1.41334964],
       [ 0.14551813,  1.25859291]])

In [55]:
df = k_means_clustering(k=4)

Opening data from local file.
Cluster 1 Mean:
customer_age                       28.105851
sub_category_Bike Stands            0.001808
sub_category_Bottles and Cages      0.254438
sub_category_Caps                   0.076677
sub_category_Cleaners               0.024737
sub_category_Fenders                0.019888
sub_category_Gloves                 0.012410
sub_category_Helmets                0.048159
sub_category_Hydration Packs        0.004356
sub_category_Jerseys                0.024901
sub_category_Mountain Bikes         0.003698
sub_category_Road Bikes             0.001890
sub_category_Shorts                 0.005999
sub_category_Socks                  0.017751
sub_category_Tires and Tubes        0.499178
sub_category_Touring Bikes          0.000247
sub_category_Vests                  0.002794
revenue                           179.097304
dtype: float64

Cluster 2 Mean:
customer_age                       48.486637
sub_category_Bike Stands            0.003155
sub_category_Bottles a

,date,year,month,customer_age,customer_gender,country,state,product_category,sub_category,quantity,...,sub_category_Socks,sub_category_Tires and Tubes,sub_category_Touring Bikes,sub_category_Vests,customer_gender_M,country_Germany,country_United Kingdom,country_United States,product_category_encoded,clusters
0,2016-02-19,2016,February,29,F,United States,Washington,Accessories,Tires and Tubes,1,...,0,1,0,0,0,0,0,1,0,0
1,2016-02-20,2016,February,29,F,United States,Washington,Clothing,Gloves,2,...,0,0,0,0,0,0,0,1,2,0
2,2016-02-27,2016,February,29,F,United States,Washington,Accessories,Tires and Tubes,3,...,0,1,0,0,0,0,0,1,0,0
3,2016-03-12,2016,March,29,F,United States,Washington,Accessories,Tires and Tubes,2,...,0,1,0,0,0,0,0,1,0,0
4,2016-03-12,2016,March,29,F,United States,Washington,Accessories,Tires and Tubes,3,...,0,1,0,0,0,0,0,1,0,0


In [27]:
df, train, validate, test = w.wrangle_data()
    
train_scaled, validate_scaled, test_scaled = scale_data(train, validate, test, columns_to_scale=['customer_age',
                                                                                                    'unit_cost', 
                                                                                                     'unit_price',
                                                                                                     'cost',
                                                                                                     'revenue',
                                                                                                     'profit'])

    
modeling_feats = ['customer_age','sub_category', 'revenue']  # Select relevant columns
    
train_clustering = train_scaled[modeling_feats]
validate_clustering = validate_scaled[modeling_feats]
test_clustering = test_scaled[modeling_feats]

train_standardized = (train_clustering - train_clustering.mean()) / train_clustering.std()
validate_standardized = (validate_clustering - validate_clustering.mean()) / validate_clustering.std()
test_standardized = (test_clustering - test_clustering.mean()) / test_clustering.std()

# Clustering Algorithm (K-means)
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans = kmeans.fit_transform(train_clustering)
train_labels = kmeans.predict(train_clustering)
    

Opening data from local file.


/var/folders/gn/krty7k0175z7_1gpw49cwdch0000gn/T/ipykernel_4249/3895181490.py:14: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_standardized = (train_scaled - train_scaled.mean()) / train_scaled.std()
/var/folders/gn/krty7k0175z7_1gpw49cwdch0000gn/T/ipykernel_4249/3895181490.py:14: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_standardized = (train_scaled - train_scaled.mean()) / train_scaled.std()
/var/folders/gn/krty7k0175z7_1gpw49cwdch0000gn/T/ipykernel_4249/3895181490.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select

In [29]:
train_scaled.head()

,year,month,customer_age,customer_gender,country,state,product_category,sub_category,quantity,unit_cost,...,sub_category_Shorts,sub_category_Socks,sub_category_Tires and Tubes,sub_category_Touring Bikes,sub_category_Vests,customer_gender_M,country_Germany,country_United Kingdom,country_United States,product_category_encoded
8366,2016,June,-0.572424,M,United Kingdom,England,Accessories,Tires and Tubes,2,-0.590322,...,0,0,1,0,0,1,0,1,0,0
3413,2016,January,-1.021137,F,United Kingdom,England,Accessories,Helmets,2,0.259027,...,0,0,0,0,0,0,0,1,0,0
11230,2016,March,-0.931394,F,United States,California,Accessories,Bottles and Cages,2,-0.538846,...,0,0,0,0,0,0,0,0,1,0
29743,2015,November,3.824962,M,United States,California,Clothing,Vests,2,-0.386478,...,0,0,0,0,1,1,0,0,1,2
3176,2015,September,0.863457,F,France,Essonne,Bikes,Road Bikes,1,4.316339,...,0,0,0,0,0,0,0,0,0,1


In [32]:
train.columns

Index(['date', 'year', 'month', 'customer_age', 'customer_gender', 'country',
       'state', 'product_category', 'sub_category', 'quantity', 'unit_cost',
       'unit_price', 'cost', 'revenue', 'profit', 'sub_category_Bike Stands',
       'sub_category_Bottles and Cages', 'sub_category_Caps',
       'sub_category_Cleaners', 'sub_category_Fenders', 'sub_category_Gloves',
       'sub_category_Helmets', 'sub_category_Hydration Packs',
       'sub_category_Jerseys', 'sub_category_Mountain Bikes',
       'sub_category_Road Bikes', 'sub_category_Shorts', 'sub_category_Socks',
       'sub_category_Tires and Tubes', 'sub_category_Touring Bikes',
       'sub_category_Vests', 'customer_gender_M', 'country_Germany',
       'country_United Kingdom', 'country_United States',
       'product_category_encoded'],
      dtype='object')

In [ ]:
df.info()

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
# X = train[['customer_age', 'quantity', 'unit_cost',
#        'unit_price', 'cost', 'revenue', 'total_price',
#        'sub_category_Bike Stands', 'sub_category_Bottles and Cages',
#        'sub_category_Caps', 'sub_category_Cleaners', 'sub_category_Fenders',
#        'sub_category_Gloves', 'sub_category_Helmets',
#        'sub_category_Hydration Packs', 'sub_category_Jerseys',
#        'sub_category_Mountain Bikes', 'sub_category_Road Bikes',
#        'sub_category_Shorts', 'sub_category_Socks',
#        'sub_category_Tires and Tubes', 'sub_category_Touring Bikes',
#        'sub_category_Vests', 'customer_gender_M', 'country_Germany',
#        'country_United Kingdom', 'country_United States']]

In [ ]:
# {k: KMeans(k).fit(X
#                  ).inertia_ for k in range(2,20)}
# # alternatively:
# my_kmeans_dict = {}
# for k in range(1,15):
#     my_kmeans_dict[k] = KMeans(k).fit(X).inertia_

In [ ]:
# plt.figure(figsize=(12,12))
# pd.Series(my_kmeans_dict).plot(marker='x')
# plt.title('Elbow! Elbow!')
# plt.show()

In [ ]:
# # we preprocessed! huzzah!
# # lets make a clustering object from sklearn
# # Make a thing! Thats my favorite!
# k_means_prototype = KMeans(n_clusters=5)
# # fit the thing!!!!
# k_means_prototype.fit(X)
# # use the thing
# clusters = k_means_prototype.predict(X)

In [ ]:
# train['clusters'] = clusters

In [ ]:
# train.clusters.value_counts()

In [ ]:
# X = train[['customer_age', 'quantity', 'revenue', 
#            'customer_gender_M', 'country_Germany',
#            'country_United Kingdom', 'country_United States']]

In [ ]:
# {k: KMeans(k).fit(X
#                  ).inertia_ for k in range(2,20)}
# # alternatively:
# my_kmeans_dict = {}
# for k in range(1,15):
#     my_kmeans_dict[k] = KMeans(k).fit(X).inertia_

In [ ]:
# plt.figure(figsize=(12,12))
# pd.Series(my_kmeans_dict).plot(marker='x')
# plt.title('Elbow! Elbow!')
# plt.show()

In [ ]:
# train.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

In [ ]:
# train.columns

In [ ]:
# train_ex = train[['date', 'year', 'month', 'customer_age', 'customer_gender', 'country',
#        'state', 'product_category', 'sub_category', 'quantity', 'unit_cost',
#        'unit_price', 'cost', 'revenue', 'total_price', 'clusters']]

In [ ]:
def univariate_visulization(df):
    
    cat_cols, num_cols = [], []
    for col in df.columns:
        if df[col].dtype == "o":
            cat_cols.append(col)
        else:
            if df[col].nunique() < 5:
                cat_cols.append(col)
            else: 
                num_cols.append(col)
                
    explore_cols = cat_cols + num_cols

    for col in explore_cols:
        
        if col in cat_cols:
            print(f'Univariate assessment of feature {col}:')
            sns.countplot(data=df, x=col, color='violet', edgecolor='black')
            plt.xticks(rotation=45)
            plt.show()

        if col in num_cols:
            print(f'Univariate feature analysis of feature {col}: ')
            plt.hist(df[col], color='violet', edgecolor='black')
            plt.show()
            plt.xticks(rotation=45)
            df[col].describe()
    print('_____________________________________________________')
    print('_____________________________________________________')
    print()

In [ ]:
def bivariate_visulization(df, target):
    
    cat_cols = ['year', 'month', 'customer_gender', 'country',
       'state', 'product_category', 'sub_category', 'quantity', 'clusters']
    num_cols = ['customer_age', 'unit_cost',
       'unit_price', 'cost', 'revenue', 'total_price'] 
        
    print(f'Numeric Columns: {num_cols}')
    print(f'Categorical Columns: {cat_cols}')
    explore_cols = cat_cols + num_cols

    for col in explore_cols:
        if col in cat_cols:
            if col != target:
                print(f'Bivariate assessment of feature {col}:')
                sns.barplot(data = df, y = df[col], x = df[target], palette='crest')
                plt.xticks(rotation=45)
                plt.show()

#         if col in num_cols:
#             if col != target:
#                 print(f'Bivariate feature analysis of feature {col}: ')
#                 plt.scatter(y = df[col], x = df[target], color='turquoise')
#                 plt.axhline(df[target].mean(), ls=':', color='red')
#                 plt.axvline(df[col].mean(), ls=':', color='red')
#                 plt.xticks(rotation=45)
#                 plt.show()

    print('_____________________________________________________')
    print('_____________________________________________________')
    print()

> Whats the ratio between costs and price on average

In [ ]:
df = w.k_means_clustering()

In [ ]:
explore_cols = ['date', 'year', 'month', 'customer_age', 'customer_gender', 'country',
       'state', 'product_category', 'sub_category', 'quantity', 'unit_cost',
       'unit_price', 'cost', 'revenue', 'total_price', 'clusters']

cluster_0 = train[train.clusters == 0][explore_cols]
cluster_1 = train[train.clusters == 1][explore_cols]
cluster_2 = train[train.clusters == 2][explore_cols]
cluster_3 = train[train.clusters == 3][explore_cols]
cluster_4 = train[train.clusters == 4][explore_cols]  

In [ ]:
clusters = [cluster_0, cluster_1, cluster_2, cluster_3, cluster_4]

In [ ]:
fig, axs = plt.subplots(len(clusters), 1, figsize=(15, 30))

axs[0].hist(cluster_0.customer_age, color='cadetblue', ec='black')
axs[1].hist(cluster_1.customer_age, color='darkturquoise', ec='black')
axs[2].hist(cluster_2.customer_age, color='skyblue', ec='black')
axs[3].hist(cluster_3.customer_age, color='teal', ec='black')
axs[4].hist(cluster_4.customer_age, color='mediumaquamarine', ec='black')

fig.suptitle("Comparing the Distribution of Customer Age Across Clusters")
axs[0].set_title('Cluster 0 Customer Age Distribution')
axs[1].set_title('Cluster 1 Customer Age Distribution')
axs[2].set_title('Cluster 2 Customer Age Distribution')
axs[3].set_title('Cluster 3 Customer Age Distribution')
axs[4].set_title('Cluster 4 Customer Age Distribution')

axs[0].set_xlabel('Age Brackets')
axs[0].set_ylabel('Count')
axs[1].set_xlabel('Age Brackets')
axs[1].set_ylabel('Count')
axs[2].set_xlabel('Age Brackets')
axs[2].set_ylabel('Count')
axs[3].set_xlabel('Age Brackets')
axs[3].set_ylabel('Count')
axs[4].set_xlabel('Age Brackets')
axs[4].set_ylabel('Count')

plt.show()

In [ ]:
def get_count_gender():
    ''' Gets boxplots of acquired continuous variables'''

    plt.figure(figsize=(5, 25))

    for i, cluster in enumerate(clusters):
        
        plot_number = i + 1 
        
        # Create subplot.
        plt.subplot(len(clusters), 1, plot_number)

        # Title with column name.
        plt.title(f'Comparing Counts of Transactions by Gender for cluster_{i}')

        # Display boxplot for column.
        sns.countplot(x=cluster['customer_gender'], hue=cluster['product_category'], palette='crest')

        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        
        # Hide gridlines.
        plt.grid(False)

    plt.show()

In [ ]:
get_count_gender()

In [ ]:
train.groupby('customer_gender').revenue.sum()

In [ ]:
train.groupby(['clusters', 'customer_gender']).revenue.mean()

In [ ]:
train.groupby(['clusters', 'product_category']).revenue.mean()

In [ ]:
train.groupby(['clusters', 'product_category']).revenue.mean()

In [ ]:
cluster_3 = train[train.clusters == 3]

In [ ]:
cluster_3

In [ ]:
cluster_1.groupby('sub_category').revenue.mean()

In [ ]:
cluster_3.groupby('sub_category').revenue.mean()

In [ ]:
cluster_1.groupby('customer_gender').revenue.mean()

In [ ]:
cluster_3.groupby('customer_gender').revenue.mean()

In [ ]:
import cbm_explore as ex

In [ ]:
ex.viz_distribution_age()

In [ ]:
ex.get_count_gender()

In [ ]:
ex.get_count_country()

In [ ]:
def get_revenue_by_subcat():
    ''' Gets boxplots of acquired continuous variables'''

    plt.figure(figsize=(15, 30))

    for i, cluster in enumerate(clusters):
        
        plot_number = i + 1 
        
        # Create subplot.
        plt.subplot(len(clusters), 1, plot_number)

        # Title with column name.
        plt.title(f'Comparing Counts of Transactions by Country for cluster_{i}')

        # Display boxplot for column.
        sns.boxplot(x=cluster['sub_category'], y=cluster['revenue'], palette='crest')
        
        plt.xticks(rotation=45)
        
        # using padding
        plt.tight_layout(pad=5.0)
        
        # Hide gridlines.
        plt.grid(False)

    plt.show()

In [ ]:
get_revenue_by_subcat()